Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/classification-with-onnx/auto-ml-classification-with-onnx.png)

# Quiz - MNIST ONNX Model Registration and Deployment
_**Register best MNIST model trained by AutoML and deploy it for inferencing**_

## Introduction
In this notebook, you need to register the MNIST model you trained with AutoML in [notebook 5](https://github.com/henry-zeng/AutoML-AIConf-2019-BJ/blob/master/5_automl-classification-with-onnx.ipynb) and using what you learned in [notebook 6](https://github.com/henry-zeng/AutoML-AIConf-2019-BJ/blob/master/6_onnx-inference-mnist-deploy.ipynb) to deploy in ACI for inferencing. Below lists the key steps you need to complete:

1. Setup
2. Register your model with Azure ML
3. Specify score and environment files
4. Write docker file and create container image
5. Deploy

After you complete these steps, you can use the sample code we provide to verify and test the deployed web service.

## Setup

As part of the setup you have already created an Azure ML `Workspace` object. For AutoML you will need to create an `Experiment` object, which is a named object in a `Workspace` used to run experiments.

In [ ]:
import logging
import os

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

In [ ]:
ws = Workspace.from_config()

# Choose a name for the experiment and specify the project folder.
experiment_name = 'automl-classification-onnx'
project_folder = './projects/automl-classification-onnx'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

### Register your model with Azure ML

In [ ]:
# Reference code in notebook 6: https://github.com/henry-zeng/AutoML-AIConf-2019-BJ/blob/master/6_onnx-inference-mnist-deploy.ipynb


### Verify the registered model is working

In [ ]:
# You can use the following code to verify if your registered model is working
'''
import numpy as np
import onnxruntime
from azureml.core.model import Model

session = onnxruntime.InferenceSession(Model.get_model_path(model_name = 'mnist_best_onnx'), None)

print(Model.get_model_path(model_name = 'mnist_best_onnx'))

k = 66
X_test_k = X_test[k].tolist()
y_test_k = y_test[k]

print("Ground truth:", y_test_k)

session_input = {}
for x in session.get_inputs():
    i = int(x.name[1:]) - 1 # x.name should be 'CXXX', in which 'XXX' indicates the column index starting from 1.
    session_input[x.name] = np.array([X_test_k[i]]).astype('float32')

r = session.run([session.get_outputs()[0].name], session_input)

print("Prediction:  ", r[0][0])
'''

### Specify our score and environment files

We are now going to deploy our ONNX Model on AML with inference in ONNX Runtime. We begin by writing a score.py file, which will help us run the model in our Azure ML virtual machine (VM), and then specify our environment by writing a yml file. You will also notice that we import the onnxruntime library to do runtime inference on our ONNX models (passing in input and evaluating out model's predicted output). More information on the API and commands can be found in the [ONNX Runtime documentation](https://aka.ms/onnxruntime).

### Write score file

A score file is what tells our Azure cloud service what to do. After initializing our model using azureml.core.model, we start an ONNX Runtime inference session to evaluate the data passed in on our function calls.

In [ ]:
# The score file below is a reference for you.
'''
%%writefile score.py
import json
import numpy as np
import onnxruntime
import sys
import os
from azureml.core.model import Model
import time


def init():
    global session, input_name, output_name
    model = Model.get_model_path(model_name = 'mnist_best_onnx')
    session = onnxruntime.InferenceSession(model, None)


def run(input_data):

    try:
        # load in our data, convert to readable format
        data = np.array(json.loads(input_data)['data']).astype('float32').tolist()
        session_input = {}
        for x in session.get_inputs():
            i = int(x.name[1:]) - 1 # x.name should be 'CXXX', in which 'XXX' indicates the column index starting from 1.
            session_input[x.name] = np.array([data[i]]).astype('float32')

        # start timer
        start = time.time()

        r = session.run([session.get_outputs()[0].name], session_input)

        #end timer
        end = time.time()

        result_dict = {"result": int(r[0][0]),
                      "time_in_sec": end - start}
    except Exception as e:
        result_dict = {"error": str(e)}

    return result_dict
'''

### Write environment file

This step creates a YAML environment file that specifies which dependencies we would like to see in our Linux Virtual Machine.

In [ ]:
# Reference code in notebook 6: https://github.com/henry-zeng/AutoML-AIConf-2019-BJ/blob/master/6_onnx-inference-mnist-deploy.ipynb


### Write Dockerfile

### Create the Container Image
This step will likely take a few minutes.

In [ ]:
# Reference code in notebook 6: https://github.com/henry-zeng/AutoML-AIConf-2019-BJ/blob/master/6_onnx-inference-mnist-deploy.ipynb


We're all done specifying what we want our virtual machine to do. Let's configure and deploy our container image.

### Deploy the container image

In [ ]:
# Reference code in notebook 6: https://github.com/henry-zeng/AutoML-AIConf-2019-BJ/blob/master/6_onnx-inference-mnist-deploy.ipynb


### Success!

If you've made it this far, you've deployed a working VM with a handwritten digit classifier running in the cloud using Azure ML. Congratulations!

You can get the URL for the webservice with the code below. Let's now see how well our model deals with our test images.

In [ ]:
# print(aci_service.scoring_uri)

### Show some sample images in X_test
We use `matplotlib` to plot 10 test images from the dataset.

In [ ]:
plt.figure(figsize = (16, 6))
for k in np.arange(10):
    plt.subplot(1, 15, k+1)
    plt.axhline('')
    plt.axvline('')
    plt.imshow(X_test[k].reshape(28, 28), cmap = plt.cm.Greys)
plt.show()

### Run evaluation / prediction

In [ ]:
plt.figure(figsize = (16, 6), frameon=False)
plt.subplot(1, 11, 1)

plt.text(x = 0, y = -30, s = "True Label: ", fontsize = 11, color = 'black')
plt.text(x = 0, y = -20, s = "Result: ", fontsize = 11, color = 'black')
plt.text(x = 0, y = -10, s = "Inference Time: ", fontsize = 11, color = 'black')
plt.text(x = 3, y = 14, s = "Model Input", fontsize = 10, color = 'black')
plt.text(x = 6, y = 18, s = "(28 x 28)", fontsize = 10, color = 'black')
plt.imshow(np.ones((28,28)), cmap=plt.cm.Greys)    


for i in np.arange(10):
    
    input_data = json.dumps({'data': X_test[i].tolist()})
    
    
    # predict using the deployed model
    r = aci_service.run(input_data)
    
    if "error" in r:
        print("Error:", r['error'])
        break
        
    result = r['result']
    time_ms = np.round(r['time_in_sec'] * 1000, 2)
    
    ground_truth = int(y_test[i])
    
    # compare actual value vs. the predicted values:
    plt.subplot(1, 11, i+2)
    plt.axhline('')
    plt.axvline('')

    # use different color for misclassified sample
    font_color = 'red' if ground_truth != result else 'black'
    clr_map = plt.cm.gray if ground_truth != result else plt.cm.Greys

    # ground truth labels are in blue
    plt.text(x = 10, y = -30, s = ground_truth, fontsize = 18, color = 'blue')
    
    # predictions are in black if correct, red if incorrect
    plt.text(x = 10, y = -20, s = result, fontsize = 18, color = font_color)
    plt.text(x = 5, y = -10, s = str(time_ms) + ' ms', fontsize = 14, color = font_color)

    
    plt.imshow(X_test[i].reshape(28, 28), cmap = clr_map)

plt.show()

### Try classifying your own images!

Create your own handwritten image and pass it into the model.

In [ ]:
# Preprocessing functions take your image and format it so it can be passed
# as input into our ONNX model

import cv2

def rgb2gray(rgb):
    """Convert the input image into grayscale"""
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def resize_img(img_to_resize):
    """Resize image to MNIST model input dimensions"""
    r_img = cv2.resize(img_to_resize, dsize=(28, 28), interpolation=cv2.INTER_AREA)
    r_img.resize((1, 1, 28, 28))
    return r_img

def preprocess(img_to_preprocess):
    """Resize input images and convert them to grayscale."""
    if img_to_preprocess.shape == (28, 28):
        img_to_preprocess.resize((1, 1, 28, 28))
        return img_to_preprocess
    
    grayscale = rgb2gray(img_to_preprocess)
    processed_img = resize_img(grayscale)
    return processed_img

In [ ]:
# Replace this string with your own path/test image
# Make sure your image is square and the dimensions are equal (i.e. 100 * 100 pixels or 28 * 28 pixels)

# Any PNG or JPG image file should work

your_test_image = "<path to file>"

# e.g. your_test_image = "C:/Users/vinitra.swamy/Pictures/handwritten_digit.png"

import matplotlib.image as mpimg

if your_test_image != "<path to file>":
    img = mpimg.imread(your_test_image)
    plt.subplot(1,3,1)
    plt.imshow(img, cmap = plt.cm.Greys)
    print("Old Dimensions: ", img.shape)
    img = preprocess(img)
    print("New Dimensions: ", img.shape)
else:
    img = None

In [ ]:
if img is None:
    print("Add the path for your image data.")
else:
    input_data = json.dumps({'data': img.tolist()})

    try:
        r = aci_service.run(input_data)
        result = r['result']
        time_ms = np.round(r['time_in_sec'] * 1000, 2)
    except KeyError as e:
        print(str(e))

    plt.figure(figsize = (16, 6))
    plt.subplot(1, 15,1)
    plt.axhline('')
    plt.axvline('')
    plt.text(x = -100, y = -20, s = "Model prediction: ", fontsize = 14)
    plt.text(x = -100, y = -10, s = "Inference time: ", fontsize = 14)
    plt.text(x = 0, y = -20, s = str(result), fontsize = 14)
    plt.text(x = 0, y = -10, s = str(time_ms) + " ms", fontsize = 14)
    plt.text(x = -100, y = 14, s = "Input image: ", fontsize = 14)
    plt.imshow(img.reshape(28, 28), cmap = plt.cm.gray)    

In [ ]:
# remember to delete your service after you are done using it!

# aci_service.delete()